####  Name: Nithya R
####  Rol No  : 710020104310
#### Assignment :3




# Animal Classification Using CNN Model

In [1]:
!pip install -q kaggle

In [4]:
import os

# Create a directory named .kaggle in the home directory
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)


In [ ]:
import shutil
import os

# Copy the kaggle.json file to the .kaggle directory in the home directory
shutil.copyfile('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))


In [ ]:
pwd

In [ ]:
!kaggle datasets download -d iamsouravbanerjee/animal-image-dataset-90-different-animals

In [16]:
import zipfile

# Specify the path of the zip file
zip_path = 'animal-image-dataset-90-different-animals.zip'

# Specify the directory where you want to extract the files
extract_path = 'animal-image-dataset-90-different-animals'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the files to the specified directory
    zip_ref.extractall(extract_path)

In [47]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tqdm import tqdm
import cv2
import shutil
import os

In [50]:

dataset_path = "animal-image-dataset-90-different-animals/animals/animals"
data = []
class_name = []

 #  Task 1: Data Augmentation

In [51]:
for animal in tqdm(os.listdir(dataset_path)):
    for i in range(len(os.listdir(dataset_path+ '/' + animal))):
        if i < 40:
            img = cv2.imread(dataset_path + '/' + animal + '/' + os.listdir(dataset_path+ '/' + animal)[i])
            resized_img = cv2.resize(img,(224,224))
            resized_img = resized_img / 255.0
            data.append(resized_img)
            class_name.append(animal)

data = np.array(data,dtype = 'float32')

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:37<00:00,  2.39it/s]


In [62]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = preprocessing.LabelEncoder()
le.fit(class_name)
class_names = le.classes_
class_name = le.transform(class_name)

class_name = np.array(class_name, dtype = 'uint8')
class_name = np.resize(class_name, (len(class_name),1))

##  Split the dataset into train and testing part

In [65]:
train_images, test_images, train_labels, test_labels = train_test_split(
    data, class_name, test_size=0.3, stratify = class_name
)

#  Task 2: Building CNN Model

In [25]:
#library for Model

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [26]:

input_shape = (224, 224, 3)


# First I am using ----  Sequential model 

In [ ]:
 


# Create a list of all the class names
class_name_list = os.listdir(dataset_path)

# Create the training and testing directories
os.mkdir("train")
os.mkdir("test")

# Move the images into their respective class folders
for class_name in class_name_list:
    os.mkdir(os.path.join("train", class_name))
    os.mkdir(os.path.join("test", class_name))
    images = os.listdir(os.path.join(dataset_path, class_name))
    train_images, test_images = train_test_split(images, test_size=0.2)
    for image in train_images:
        src = os.path.join(dataset_path, class_name, image)
        dst = os.path.join("train", class_name, image)
        shutil.copyfile(src, dst)
    for image in test_images:
        src = os.path.join(dataset_path, class_name, image)
        dst = os.path.join("test", class_name, image)
        shutil.copyfile(src, dst)

        

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255
)

In [ ]:
train_path = "train"

# Create the training data generator
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
     shuffle=True,
    subset='training'
)


In [ ]:
# Define the path to the training dataset
val_datagen = ImageDataGenerator(rescale=1./255)

test_path ="test"

# Create the training data generator
val_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical' 
)

In [52]:
input_shape = (224, 224, 3)
model = Sequential()

 ### 2a.     Input layer

In [53]:
# input layer
model.add(Conv2D(224, (3, 3), activation="relu", input_shape=input_shape))

 ### 2b.    1 Convolution & 1 Pooling layer must to present task given

In [54]:

model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

### 2c. 1 Flatten layer

In [55]:
# flatten layer
model.add(Flatten())

### 2d.  Hidden layer

In [56]:
# first hidden layer
model.add(Dense(128, activation="relu"))

# second hidden layer
model.add(Dense(64, activation="relu"))

### 2e. Output layer

In [57]:
# output layer
model.add(Dense(90, activation="softmax"))

## Compile the model

In [58]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

## Train the model

In [59]:
# checking before training the mode graphic card is ON or OFF
!nvidia-smi

Sun May 14 00:00:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX450          WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P0               N/A /  N/A|      0MiB /  2048MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [60]:

import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [61]:
history = model.fit(
    steps_per_epoch=32,
    epochs=30,
    validation_data=val_generator,
    validation_steps=32
)

Epoch 1/30
32/32 [==============================] - 368s 11s/step - loss: 4.6257 - accuracy: 0.0078
Epoch 2/30
32/32 [==============================] - 374s 12s/step - loss: 4.4971 - accuracy: 0.0117
Epoch 3/30
32/32 [==============================] - 403s 13s/step - loss: 4.5000 - accuracy: 0.0137
Epoch 4/30
32/32 [==============================] - 436s 14s/step - loss: 4.5003 - accuracy: 0.0088
Epoch 5/30
32/32 [==============================] - 437s 14s/step - loss: 4.5002 - accuracy: 0.0166
Epoch 6/30
32/32 [==============================] - 421s 13s/step - loss: 4.5000 - accuracy: 0.0146
Epoch 7/30
32/32 [==============================] - 404s 13s/step - loss: 4.5007 - accuracy: 0.0059
Epoch 8/30
32/32 [==============================] - 387s 12s/step - loss: 4.5009 - accuracy: 0.0078
Epoch 9/30
32/32 [==============================] - 388s 12s/step - loss: 4.5000 - accuracy: 0.0088
Epoch 10/30
32/32 [==============================] - 396s 12s/step - loss: 4.4994 - accuracy: 0.0127

## it's taken 3hours and 30 minute to complete 30 epochs

## Saving the model

In [62]:
# Save the model
model.save('animal_cnn.h5')

In [ ]:
import matplotlib.pyplot as plt



# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt


# 3.Testing the model

In [71]:
import cv2
import numpy as np

In [70]:
# Load the saved model
loaded_model = load_model('animal_cnn.h5')

In [93]:
# Load the image to test
img = cv2.imread('eb30b9092cf7063ed1584d05fb1d4e9fe777ead218ac104497f5c978a4efbcb0_640.jpg')

In [96]:
from PIL import Image

# Load the image
img = Image.open('eb30b9092cf7063ed1584d05fb1d4e9fe777ead218ac104497f5c978a4efbcb0_640.jpg')

# Resize the image to the same size used during training
img = img.resize((224, 224))


In [97]:

img_array = np.array(img)

# Reshape the array to have a batch dimension of 1
img_array = img_array.reshape((1, 224, 224, 3))

# Scale the pixel values to be between 0 and 1
img_array = img_array / 255.0

In [99]:
predictions = model.predict(img_array)

# Get the index of the predicted class
predicted_class_index = np.argmax(predictions)

# Invert the class_indices dictionary to get the actual class name
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}
predicted_class_name = class_names[predicted_class_index]

print('Result    ', predicted_class_name)

1/1 [==============================] - 0s 99ms/step
Result     fly
